In [ ]:
from env.core import FunctionExpr, Param, AndNode, OrNode, IntBooleanNode
from env.full_state import FullState
from env.meta_env import MetaInfo
from env.goal_env import GoalEnv
from env.node_types import HaveDefinition

params = (Param.from_int(1), Param.from_int(2), Param.from_int(3))
p1, p2, p3 = params
goal = HaveDefinition(
    FunctionExpr.with_child(
        OrNode(
            AndNode(p1, p2, IntBooleanNode(1)),
            AndNode(p2, p3),
        ),
    )
)

env = GoalEnv(goal)
node_types = env.full_state.node_types()
print('Amount of node types:', len(node_types))

In [167]:
# selected_types = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_all_types)).apply()
# GoalEnv.symbolic(selected_types, node_types)

In [168]:
selected_goal = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_goal)).apply()
assert selected_goal == goal

In [ ]:
GoalEnv.symbolic(selected_goal, node_types)

In [ ]:
GoalEnv.data_array(selected_goal, node_types)

In [ ]:
env.to_symbolic_current_state()

In [ ]:
env.to_data_array_current_state()

In [ ]:
len(env.to_data_array())

In [174]:
# details = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_all_types_details)).apply()
# selected = details.nested_arg(102).apply()
# GoalEnv.symbolic(selected, node_types)

In [175]:
# import sympy
# sympy.latex(GoalEnv.symbolic(selected, node_types))

In [176]:
# GoalEnv.data_array(selected, node_types)

In [177]:
# env.to_symbolic()

In [178]:
env.full_state.validate()

In [ ]:
from env.full_state import HistoryNode
def get_current_state(env: GoalEnv):
    return env.full_state.nested_args((FullState.idx_current, HistoryNode.idx_state)).apply()
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [180]:
from env.state import State
assert current_state == State.create()

In [ ]:
from env.action_impl import CreateScratch
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [182]:
assert current_state == State.from_raw(
    scratchs=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.full_state import HistoryGroupNode
from env.core import Optional
def get_last_history_meta(env: GoalEnv):
    history = env.full_state.history.apply().cast(HistoryGroupNode)
    last = history.as_tuple[-1]
    return last.meta_data.apply().nested_arg(Optional.idx_value).apply()
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [184]:
from env.action import ActionData
from env.action_impl import CreateScratchOutput
from env.state import StateScratchIndex

assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(1),
        Optional(),
    )),
    exception=Optional(),
)

In [185]:
# GoalEnv.symbolic(selected, node_types)

In [186]:
# GoalEnv.symbolic(env.full_state.history.apply(), node_types)

In [187]:
# selected_types = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_from_int_group)).apply()
# GoalEnv.symbolic(selected_types, node_types)

In [ ]:
from env.action_impl import DefineScratchFromInt
# should return an exception (index won't return element)
action = DefineScratchFromInt.from_raw(1, len(node_types)+1, 3)
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [189]:
assert current_state == State.from_raw(
    scratchs=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [191]:
from env.action import IAction, ActionData
from env.core import BooleanExceptionInfo, IsEmpty

def get_empty_exception(action: IAction):
    return ActionData.from_args(
        action=Optional(action),
        output=Optional(),
        exception=Optional(BooleanExceptionInfo(IsEmpty(Optional()))),
    )

assert last_history_meta == get_empty_exception(action)

In [ ]:
from env.action_impl import DefineScratchFromIntIndex
from env.core import INode
from env.meta_env import SubtypeOuterGroup, GeneralTypeGroup
from env.full_state import MetaAllTypesTypeIndex

def get_info_type_index(node_type: type[INode], env: GoalEnv, node_types: tuple[type[INode], ...]):
    index_type_idx = node_types.index(MetaAllTypesTypeIndex) + 1
    type_node = node_types[index_type_idx-1].as_type()
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_full_state_int_index_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(type_node) + 1
    node_idx = node_types.index(node_type) + 1
    return meta_idx, node_idx

meta_idx, type_idx = get_info_type_index(AndNode, env, node_types)
action = DefineScratchFromIntIndex.from_raw(1, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [193]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type()]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [195]:
from env.core import Integer
from env.full_state import MetaFullStateIntIndexTypeIndex
from env.action_impl import DefineScratchOutput
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(1),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(AndNode.as_type()),
    )),
    exception=Optional(),
)

In [196]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [197]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(2),
        Optional(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [198]:
meta_idx, type_idx = get_info_type_index(OrNode, env, node_types)
action = DefineScratchFromIntIndex.from_raw(2, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [199]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type()]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(2),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(2),
        Optional(OrNode.as_type()),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [200]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [201]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(3),
        Optional(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(current_state, node_types)

In [203]:
from env.core import (
    ExtendedTypeGroup,
    RestTypeGroup,
    UnknownType,
    LaxOpaqueScope,
    OptionalValueGroup,
    ScopeId,
)
from env.state import PartialArgsGroup, StateArgsGroupIndex
from env.action_impl import CreateArgsGroupOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(),
            Optional(),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), None]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(3),
        Optional(),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(1),
        arg_group,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [204]:
from env.action_impl import DefineScratchFromInt

def get_from_int_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_from_int_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [205]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [206]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [207]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        OptionalContext(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [208]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [209]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [210]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [211]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(Param.from_int(2)),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        OptionalContext(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [212]:
meta_idx = get_from_int_type_index(IntBooleanNode, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [213]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), IntBooleanNode.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(IntBooleanNode.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [214]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 3, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [215]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(Param.from_int(2)),
            Optional(IntBooleanNode.from_int(1)),
        ),
    )
)

scratchs = tuple([AndNode.as_type(), OrNode.as_type(), IntBooleanNode.from_int(1)])
args_groups = tuple([arg_group])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        OptionalContext(IntBooleanNode.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [216]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [217]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(),
            Optional(),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(2),
        Optional(),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(2),
        arg_group_2,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [218]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [219]:
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [220]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [221]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(2)),
            Optional(),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        OptionalContext(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [222]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [223]:
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(3)])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(3),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [224]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [225]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(2)),
            Optional(Param.from_int(3)),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        OptionalContext(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [226]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(3, 1, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [227]:
call_1 = AndNode(
    Param.from_int(1),
    Param.from_int(2),
    IntBooleanNode.from_int(1),
)
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(3),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(1)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [228]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 1, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [229]:
call_2 = AndNode(
    Param.from_int(2),
    Param.from_int(3),
)
scratchs = tuple([call_2, OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [230]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [231]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(call_1),
            Optional(Param.from_int(3)),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        OptionalContext(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [232]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [233]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(call_1),
            Optional(call_2),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(1),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        OptionalContext(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [234]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 2, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [235]:
call_3 = OrNode(
    call_1,
    call_2,
)
scratchs = tuple([call_3, OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(2),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(call_3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
GoalEnv.symbolic(current_state, node_types)

In [ ]:
GoalEnv.data_array(current_state, node_types)

In [238]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [239]:
args_groups = tuple([arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [240]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [241]:
args_groups = tuple()

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [242]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [243]:
scratchs = tuple([call_3, OrNode.as_type()])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [244]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [245]:
scratchs = tuple([call_3])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [246]:
from env.core import FunctionExpr
from env.action_impl import DefineScratchFromSingleArg

def get_single_child_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_single_child_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_single_child_type_index(FunctionExpr, env)
action = DefineScratchFromSingleArg.from_raw(1, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [ ]:
GoalEnv.symbolic(current_state, node_types)

In [ ]:
GoalEnv.data_array(current_state, node_types)

In [249]:
from env.state import Scratch
content = current_state.nested_args((
    State.idx_scratch_group,
    1,
    Scratch.idx_child,
    Optional.idx_value,
)).apply()
assert content == goal.nested_arg(1).apply()